In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 

In [2]:
import numpy as np
import tensorflow as tf

from tensorflow.keras import Sequential
from tensorflow.keras import Model
from tensorflow.keras.layers import (RandomFlip, RandomRotation, RandomContrast, 
                                    GlobalAveragePooling2D, Dense, BatchNormalization)
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.keras.applications.efficientnet_v2 import EfficientNetV2B2
from tensorflow.keras.optimizers import SGD

2024-03-13 07:51:50.347160: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-13 07:51:50.347179: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-13 07:51:50.347666: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
seed = 42
dataset_path = '/home/deeplearning/AI/datasets/TrashBox'
epochs = 30
batch_size = 32
img_height = 224
img_width = 224

In [4]:
data_augmentation = Sequential([
    RandomFlip(mode="horizontal", seed=seed),
    RandomRotation(factor=0.05, seed=seed),
    RandomContrast(factor=0.2, seed=seed)
])

In [5]:
train_ds_raw = image_dataset_from_directory(dataset_path + '/train/sub_classes', batch_size=batch_size)
train_ds = train_ds_raw.map(lambda img, label: (data_augmentation(img), label)).shuffle(100, seed=seed).prefetch(tf.data.AUTOTUNE)

val_ds = image_dataset_from_directory(dataset_path + '/val', batch_size=batch_size)

Found 12999 files belonging to 25 classes.
Found 1642 files belonging to 25 classes.


In [6]:
class_names = train_ds_raw.class_names
n_classes = len(class_names)

In [7]:
def train(lr, weight_decay, max_epochs):
    print(f'lr: {lr:.6f}, weight_decay:{weight_decay:.6f}')
    base_model = EfficientNetV2B2(weights="imagenet", include_top=False)
    avg = GlobalAveragePooling2D()(base_model.output)
    output = Dense(n_classes, activation="softmax")(avg)
    model = Model(inputs=base_model.input, outputs=output)

    for layer in model.layers:
        if isinstance(layer, BatchNormalization):
            layer.momentum = 0.98 # max(1 − 10/s, 0.9), where s is the number of steps per epoch, for this dataset and batch size, s= 407r

    optimizer = SGD(learning_rate=lr, momentum=0.9, nesterov=True, weight_decay=weight_decay)
    model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
                  metrics=["accuracy"])
    history = model.fit(train_ds, validation_data=val_ds, epochs=max_epochs)

    return history

In [8]:
max_epochs = 10
min_lr, max_lr = -4, -1
min_wd, max_wd = -6, -3
lr_list = np.logspace(min_lr, max_lr, 7).tolist()
wd_list = [0] + np.logspace(min_wd, max_wd, 7).tolist()

In [9]:
for lr in lr_list:
    for wd in wd_list:
        train(lr, wd, max_epochs)

lr: 0.000100, weight_decay:0.000000
Epoch 1/10


2024-03-13 07:51:59.464147: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel/block1b_drop/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
I0000 00:00:1710330730.026237   42833 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


407/407 [==============================] - 56s 85ms/step - loss: 3.1687 - accuracy: 0.0909 - val_loss: 3.0820 - val_accuracy: 0.1583
Epoch 2/10
407/407 [==============================] - 42s 82ms/step - loss: 2.9418 - accuracy: 0.2334 - val_loss: 2.8582 - val_accuracy: 0.3155
Epoch 3/10
407/407 [==============================] - 42s 82ms/step - loss: 2.7139 - accuracy: 0.3367 - val_loss: 2.6315 - val_accuracy: 0.4038
Epoch 4/10
407/407 [==============================] - 42s 82ms/step - loss: 2.4897 - accuracy: 0.3905 - val_loss: 2.4173 - val_accuracy: 0.4762
Epoch 5/10
407/407 [==============================] - 42s 82ms/step - loss: 2.2997 - accuracy: 0.4393 - val_loss: 2.2211 - val_accuracy: 0.5286
Epoch 6/10
407/407 [==============================] - 42s 82ms/step - loss: 2.1417 - accuracy: 0.4886 - val_loss: 2.0480 - val_accuracy: 0.5871
Epoch 7/10
407/407 [==============================] - 42s 83ms/step - loss: 1.9939 - accuracy: 0.5393 - val_loss: 1.8886 - val_accuracy: 0.6334
Epo

2024-03-13 07:59:16.014165: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel_1/block1b_drop/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


407/407 [==============================] - 54s 84ms/step - loss: 3.1278 - accuracy: 0.0962 - val_loss: 3.0359 - val_accuracy: 0.1626
Epoch 2/10
407/407 [==============================] - 43s 83ms/step - loss: 2.8973 - accuracy: 0.2549 - val_loss: 2.8075 - val_accuracy: 0.3508
Epoch 3/10
407/407 [==============================] - 42s 83ms/step - loss: 2.6633 - accuracy: 0.3467 - val_loss: 2.5619 - val_accuracy: 0.4391
Epoch 4/10
407/407 [==============================] - 42s 83ms/step - loss: 2.4466 - accuracy: 0.4005 - val_loss: 2.3452 - val_accuracy: 0.5018
Epoch 5/10
407/407 [==============================] - 43s 83ms/step - loss: 2.2573 - accuracy: 0.4534 - val_loss: 2.1557 - val_accuracy: 0.5475
Epoch 6/10
407/407 [==============================] - 43s 83ms/step - loss: 2.0844 - accuracy: 0.5059 - val_loss: 1.9789 - val_accuracy: 0.5871
Epoch 7/10
407/407 [==============================] - 42s 83ms/step - loss: 1.9383 - accuracy: 0.5429 - val_loss: 1.8356 - val_accuracy: 0.6224
Epo

...

2024-03-13 13:35:01.770802: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel_47/block1b_drop/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


407/407 [==============================] - 54s 84ms/step - loss: 0.9678 - accuracy: 0.7247 - val_loss: 0.7346 - val_accuracy: 0.7875
Epoch 2/10
407/407 [==============================] - 42s 83ms/step - loss: 0.5413 - accuracy: 0.8378 - val_loss: 0.7630 - val_accuracy: 0.7838
Epoch 3/10
407/407 [==============================] - 43s 83ms/step - loss: 0.4081 - accuracy: 0.8738 - val_loss: 0.5231 - val_accuracy: 0.8471
Epoch 4/10
407/407 [==============================] - 43s 83ms/step - loss: 0.3115 - accuracy: 0.9019 - val_loss: 0.5903 - val_accuracy: 0.8404
Epoch 5/10
407/407 [==============================] - 43s 83ms/step - loss: 0.2776 - accuracy: 0.9132 - val_loss: 0.5860 - val_accuracy: 0.8301
Epoch 6/10
407/407 [==============================] - 43s 83ms/step - loss: 0.2278 - accuracy: 0.9251 - val_loss: 0.5078 - val_accuracy: 0.8611
Epoch 7/10
407/407 [==============================] - 43s 83ms/step - loss: 0.1863 - accuracy: 0.9374 - val_loss: 0.4957 - val_accuracy: 0.8721
Epo

2024-03-13 13:42:20.213589: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel_48/block1b_drop/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


407/407 [==============================] - 54s 84ms/step - loss: 1.7860 - accuracy: 0.4875 - val_loss: 1.5319 - val_accuracy: 0.5597
Epoch 2/10
407/407 [==============================] - 42s 83ms/step - loss: 1.1071 - accuracy: 0.6697 - val_loss: 1.0867 - val_accuracy: 0.6803
Epoch 3/10
407/407 [==============================] - 42s 82ms/step - loss: 0.8507 - accuracy: 0.7431 - val_loss: 0.8721 - val_accuracy: 0.7491
Epoch 4/10
407/407 [==============================] - 42s 83ms/step - loss: 0.6976 - accuracy: 0.7869 - val_loss: 0.9468 - val_accuracy: 0.7497
Epoch 5/10
407/407 [==============================] - 42s 83ms/step - loss: 0.6067 - accuracy: 0.8114 - val_loss: 0.8882 - val_accuracy: 0.7473
Epoch 6/10
407/407 [==============================] - 42s 82ms/step - loss: 0.5113 - accuracy: 0.8381 - val_loss: 0.8926 - val_accuracy: 0.7698
Epoch 7/10
407/407 [==============================] - 42s 82ms/step - loss: 0.4486 - accuracy: 0.8605 - val_loss: 0.6601 - val_accuracy: 0.8100
Epo

KeyboardInterrupt: 